In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.models import clone_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error


import keras.backend as K
import tensorflow as tf
import warnings
from keras.models import load_model
import os
import heapq
# 경고 비활성화
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ASAC_4기_기업프로젝트_SK하이닉스/1. 데이터 전처리/5. 최종 데이터셋(2024.00.00 기준)/df_final_(1).csv')


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/ASAC_4기_기업프로젝트_SK하이닉스/2. 모델링/LSTM_test/test_df.csv')


In [ ]:
sensor_agg_col = ['max_val_1','max_val_2','max_val_3','max_val_4','max_val_5','max_val_6','max_val_7','max_val_8','max_val_9','max_val_10','max_val_11','max_val_12','max_val_13','max_val_14','max_val_15','max_val_16','max_val_17','max_val_18','max_val_19','mean_val_1','mean_val_2','mean_val_3','mean_val_4','mean_val_5','mean_val_6','mean_val_7','mean_val_8','mean_val_9','mean_val_10','mean_val_11','mean_val_12','mean_val_13','mean_val_14','mean_val_15','mean_val_16','mean_val_17','mean_val_18','mean_val_19','median_val_1','median_val_2','median_val_3','median_val_4','median_val_5','median_val_6','median_val_7','median_val_8','median_val_9','median_val_10','median_val_11','median_val_12','median_val_13','median_val_14','median_val_15','median_val_16','median_val_17','median_val_18','median_val_19','min_val_1','min_val_2','min_val_3','min_val_4','min_val_5','min_val_6','min_val_7','min_val_8','min_val_9','min_val_10','min_val_11','min_val_12','min_val_13','min_val_14','min_val_15','min_val_16','min_val_17','min_val_18','min_val_19','range_val_1','range_val_2','range_val_3','range_val_4','range_val_5','range_val_6','range_val_7','range_val_8','range_val_9','range_val_10','range_val_11','range_val_12','range_val_13','range_val_14','range_val_15','range_val_16','range_val_17','range_val_18','range_val_19','stddev_val_1','stddev_val_2','stddev_val_3','stddev_val_4','stddev_val_5','stddev_val_6','stddev_val_7','stddev_val_8','stddev_val_9','stddev_val_10','stddev_val_11','stddev_val_12','stddev_val_13','stddev_val_14','stddev_val_15','stddev_val_16','stddev_val_17','stddev_val_18','stddev_val_19']

for i in df1['STEP_NM'].unique():
        for x in sensor_agg_col:
            df1.loc[df['STEP_NM'] == i ,f'{x}_MA']=df1.loc[df1['STEP_NM'] == i,x].rolling(window=20,min_periods = 1).mean()
original_columns = [cols for cols in df1.columns if not str(cols).endswith('_MA')] # 복사본 중에 MA로 끝나지 않는 컬럼들
for cols in original_columns:
    ma_col = f"{cols}_MA"  # 이동평균 열 이름
    diff_col = f"{cols}_Diff"  # 차이 열 이름

    if ma_col in df1.columns:  # 이동평균 열이 존재하는 경우에만 계산
        df1[diff_col] = df1[cols] - df1[ma_col]
filtered_columns = [colo for colo in df1.columns if not str(colo).endswith('_MA')]
df1 = df1[filtered_columns]


In [ ]:
df1.columns.tolist()

['Unnamed: 0',
 'WAFER_ID',
 'PROD_CD',
 'EQ_ID',
 'OPER_ID',
 'RCP_ID',
 'STEP_NM',
 'TA_tm_before',
 'start_dt_tm',
 'end_dt_tm',
 'TA_tm_after',
 'P_Time',
 'max_val_1',
 'max_val_2',
 'max_val_3',
 'max_val_4',
 'max_val_5',
 'max_val_6',
 'max_val_7',
 'max_val_8',
 'max_val_9',
 'max_val_10',
 'max_val_11',
 'max_val_12',
 'max_val_13',
 'max_val_14',
 'max_val_15',
 'max_val_16',
 'max_val_17',
 'max_val_18',
 'max_val_19',
 'mean_val_1',
 'mean_val_2',
 'mean_val_3',
 'mean_val_4',
 'mean_val_5',
 'mean_val_6',
 'mean_val_7',
 'mean_val_8',
 'mean_val_9',
 'mean_val_10',
 'mean_val_11',
 'mean_val_12',
 'mean_val_13',
 'mean_val_14',
 'mean_val_15',
 'mean_val_16',
 'mean_val_17',
 'mean_val_18',
 'mean_val_19',
 'median_val_1',
 'median_val_2',
 'median_val_3',
 'median_val_4',
 'median_val_5',
 'median_val_6',
 'median_val_7',
 'median_val_8',
 'median_val_9',
 'median_val_10',
 'median_val_11',
 'median_val_12',
 'median_val_13',
 'median_val_14',
 'median_val_15',
 'median_

In [ ]:
standard_col = ['max_val_1','max_val_2','max_val_3','max_val_4','max_val_5','max_val_6','max_val_7','max_val_8','max_val_9','max_val_10','max_val_11','max_val_12','max_val_13','max_val_14','max_val_15','max_val_16','max_val_17','max_val_18','max_val_19','mean_val_1','mean_val_2','mean_val_3','mean_val_4','mean_val_5','mean_val_6','mean_val_7','mean_val_8','mean_val_9','mean_val_10','mean_val_11','mean_val_12','mean_val_13','mean_val_14','mean_val_15','mean_val_16','mean_val_17','mean_val_18','mean_val_19','median_val_1','median_val_2','median_val_3','median_val_4','median_val_5','median_val_6','median_val_7','median_val_8','median_val_9','median_val_10','median_val_11','median_val_12','median_val_13','median_val_14','median_val_15','median_val_16','median_val_17','median_val_18','median_val_19','min_val_1','min_val_2','min_val_3','min_val_4','min_val_5','min_val_6','min_val_7','min_val_8','min_val_9','min_val_10','min_val_11','min_val_12','min_val_13','min_val_14','min_val_15','min_val_16','min_val_17','min_val_18','min_val_19','range_val_1','range_val_2','range_val_3','range_val_4','range_val_5','range_val_6','range_val_7','range_val_8','range_val_9','range_val_10','range_val_11','range_val_12','range_val_13','range_val_14','range_val_15','range_val_16','range_val_17','range_val_18','range_val_19','stddev_val_1','stddev_val_2','stddev_val_3','stddev_val_4','stddev_val_5','stddev_val_6','stddev_val_7','stddev_val_8','stddev_val_9','stddev_val_10','stddev_val_11','stddev_val_12','stddev_val_13','stddev_val_14','stddev_val_15','stddev_val_16','stddev_val_17','stddev_val_18','stddev_val_19','rep_hours','step_hours','last_hours', 'max_val_1_Diff',
 'max_val_2_Diff',
 'max_val_3_Diff',
 'max_val_4_Diff',
 'max_val_5_Diff',
 'max_val_6_Diff',
 'max_val_7_Diff',
 'max_val_8_Diff',
 'max_val_9_Diff',
 'max_val_10_Diff',
 'max_val_11_Diff',
 'max_val_12_Diff',
 'max_val_13_Diff',
 'max_val_14_Diff',
 'max_val_15_Diff',
 'max_val_16_Diff',
 'max_val_17_Diff',
 'max_val_18_Diff',
 'max_val_19_Diff',
 'mean_val_1_Diff',
 'mean_val_2_Diff',
 'mean_val_3_Diff',
 'mean_val_4_Diff',
 'mean_val_5_Diff',
 'mean_val_6_Diff',
 'mean_val_7_Diff',
 'mean_val_8_Diff',
 'mean_val_9_Diff',
 'mean_val_10_Diff',
 'mean_val_11_Diff',
 'mean_val_12_Diff',
 'mean_val_13_Diff',
 'mean_val_14_Diff',
 'mean_val_15_Diff',
 'mean_val_16_Diff',
 'mean_val_17_Diff',
 'mean_val_18_Diff',
 'mean_val_19_Diff',
 'median_val_1_Diff',
 'median_val_2_Diff',
 'median_val_3_Diff',
 'median_val_4_Diff',
 'median_val_5_Diff',
 'median_val_6_Diff',
 'median_val_7_Diff',
 'median_val_8_Diff',
 'median_val_9_Diff',
 'median_val_10_Diff',
 'median_val_11_Diff',
 'median_val_12_Diff',
 'median_val_13_Diff',
 'median_val_14_Diff',
 'median_val_15_Diff',
 'median_val_16_Diff',
 'median_val_17_Diff',
 'median_val_18_Diff',
 'median_val_19_Diff',
 'min_val_1_Diff',
 'min_val_2_Diff',
 'min_val_3_Diff',
 'min_val_4_Diff',
 'min_val_5_Diff',
 'min_val_6_Diff',
 'min_val_7_Diff',
 'min_val_8_Diff',
 'min_val_9_Diff',
 'min_val_10_Diff',
 'min_val_11_Diff',
 'min_val_12_Diff',
 'min_val_13_Diff',
 'min_val_14_Diff',
 'min_val_15_Diff',
 'min_val_16_Diff',
 'min_val_17_Diff',
 'min_val_18_Diff',
 'min_val_19_Diff',
 'range_val_1_Diff',
 'range_val_2_Diff',
 'range_val_3_Diff',
 'range_val_4_Diff',
 'range_val_5_Diff',
 'range_val_6_Diff',
 'range_val_7_Diff',
 'range_val_8_Diff',
 'range_val_9_Diff',
 'range_val_10_Diff',
 'range_val_11_Diff',
 'range_val_12_Diff',
 'range_val_13_Diff',
 'range_val_14_Diff',
 'range_val_15_Diff',
 'range_val_16_Diff',
 'range_val_17_Diff',
 'range_val_18_Diff',
 'range_val_19_Diff',
 'stddev_val_1_Diff',
 'stddev_val_2_Diff',
 'stddev_val_3_Diff',
 'stddev_val_4_Diff',
 'stddev_val_5_Diff',
 'stddev_val_6_Diff',
 'stddev_val_7_Diff',
 'stddev_val_8_Diff',
 'stddev_val_9_Diff',
 'stddev_val_10_Diff',
 'stddev_val_11_Diff',
 'stddev_val_12_Diff',
 'stddev_val_13_Diff',
 'stddev_val_14_Diff',
 'stddev_val_15_Diff',
 'stddev_val_16_Diff',
 'stddev_val_17_Diff',
 'stddev_val_18_Diff',
 'stddev_val_19_Diff']


In [ ]:
cor_list = []
for i in df1['TA_tm_before'].unique():
    temp = df1[df1['TA_tm_before'] == i]
    cor = temp[standard_col].corrwith(temp['diff_hours'])
    cor = cor.abs().sort_values(ascending=False)
    cor_list.append(cor)

In [ ]:
high_corr_columns = set()
for cor in cor_list:
    high_corr_columns.update(cor[cor.abs() >= 0.5].index)
print((len(list(high_corr_columns))))
print(list(high_corr_columns))

103
['max_val_15', 'max_val_13', 'min_val_16', 'min_val_15_Diff', 'median_val_14_Diff', 'min_val_13', 'min_val_12', 'mean_val_9', 'mean_val_14_Diff', 'median_val_7', 'min_val_19', 'range_val_15', 'median_val_1', 'mean_val_3_Diff', 'mean_val_7_Diff', 'median_val_12_Diff', 'rep_hours', 'mean_val_12', 'mean_val_18', 'stddev_val_14', 'median_val_12', 'mean_val_3', 'min_val_4_Diff', 'max_val_9_Diff', 'mean_val_15', 'range_val_14', 'mean_val_15_Diff', 'max_val_9', 'min_val_2_Diff', 'range_val_15_Diff', 'max_val_12_Diff', 'min_val_1', 'median_val_19', 'min_val_13_Diff', 'min_val_3_Diff', 'max_val_16', 'min_val_4', 'stddev_val_13', 'min_val_14', 'mean_val_1_Diff', 'stddev_val_12', 'mean_val_4_Diff', 'max_val_18', 'mean_val_19', 'max_val_8', 'min_val_1_Diff', 'mean_val_12_Diff', 'median_val_4', 'mean_val_14', 'min_val_18_Diff', 'median_val_3', 'mean_val_7', 'min_val_8', 'range_val_18_Diff', 'median_val_9_Diff', 'step_hours', 'max_val_19', 'range_val_13', 'mean_val_2_Diff', 'median_val_8', 'min_

In [ ]:
[ 'min_val_15_Diff', 'median_val_14_Diff', 'mean_val_14_Diff', 'mean_val_3_Diff', 'mean_val_7_Diff', 'median_val_12_Diff', 'min_val_4_Diff', 'max_val_9_Diff', 'mean_val_15_Diff','min_val_2_Diff', 'range_val_15_Diff', 'max_val_12_Diff',   'min_val_13_Diff', 'min_val_3_Diff', 'mean_val_1_Diff', 'mean_val_4_Diff', 'max_val_18', 'mean_val_19', 'max_val_8', 'min_val_1_Diff', 'mean_val_12_Diff', 'median_val_4', 'mean_val_14', 'min_val_18_Diff', 'median_val_3', 'mean_val_7', 'min_val_8', 'range_val_18_Diff', 'median_val_9_Diff', 'step_hours', 'max_val_19', 'range_val_13', 'mean_val_2_Diff', 'median_val_8', 'min_val_3', 'stddev_val_9', 'min_val_9_Diff', 'mean_val_13_Diff', 'max_val_15_Diff', 'median_val_4_Diff', 'max_val_19_Diff', 'mean_val_9_Diff', 'median_val_2_Diff', 'median_val_15', 'mean_val_18_Diff', 'min_val_18', 'stddev_val_15', 'last_hours', 'median_val_9', 'max_val_14', 'max_val_14_Diff', 'min_val_15', 'median_val_7_Diff', 'median_val_1_Diff', 'max_val_13_Diff', 'median_val_18_Diff', 'stddev_val_9_Diff', 'median_val_3_Diff', 'range_val_13_Diff', 'max_val_18_Diff', 'stddev_val_13_Diff', 'mean_val_4', 'mean_val_8', 'min_val_9', 'median_val_15_Diff', 'mean_val_2', 'median_val_18', 'mean_val_16', 'median_val_13_Diff', 'stddev_val_18_Diff', 'stddev_val_15_Diff', 'median_val_13', 'max_val_12', 'mean_val_13', 'median_val_14', 'mean_val_1', 'median_val_2']

In [ ]:
'max_val_15', 'max_val_13', 'min_val_16', 'min_val_13', 'min_val_12', 'mean_val_9','median_val_7', 'min_val_19', 'range_val_15', 'median_val_1','rep_hours','mean_val_12', 'mean_val_18', 'stddev_val_14', 'median_val_12', 'mean_val_3', 'mean_val_15', 'range_val_14','max_val_9','min_val_1','median_val_19', 'max_val_16', 'min_val_4','stddev_val_13', 'min_val_14','stddev_val_12',

## X,y split

In [ ]:
def xy_split1(df):
    global standard_li,drop_col,step_li,diff_li
    sensor_agg_col = ['max_val_1','max_val_2','max_val_3','max_val_4','max_val_5','max_val_6','max_val_7','max_val_8','max_val_9','max_val_10','max_val_11','max_val_12','max_val_13','max_val_14','max_val_15','max_val_16','max_val_17','max_val_18','max_val_19','mean_val_1','mean_val_2','mean_val_3','mean_val_4','mean_val_5','mean_val_6','mean_val_7','mean_val_8','mean_val_9','mean_val_10','mean_val_11','mean_val_12','mean_val_13','mean_val_14','mean_val_15','mean_val_16','mean_val_17','mean_val_18','mean_val_19','median_val_1','median_val_2','median_val_3','median_val_4','median_val_5','median_val_6','median_val_7','median_val_8','median_val_9','median_val_10','median_val_11','median_val_12','median_val_13','median_val_14','median_val_15','median_val_16','median_val_17','median_val_18','median_val_19','min_val_1','min_val_2','min_val_3','min_val_4','min_val_5','min_val_6','min_val_7','min_val_8','min_val_9','min_val_10','min_val_11','min_val_12','min_val_13','min_val_14','min_val_15','min_val_16','min_val_17','min_val_18','min_val_19','range_val_1','range_val_2','range_val_3','range_val_4','range_val_5','range_val_6','range_val_7','range_val_8','range_val_9','range_val_10','range_val_11','range_val_12','range_val_13','range_val_14','range_val_15','range_val_16','range_val_17','range_val_18','range_val_19','stddev_val_1','stddev_val_2','stddev_val_3','stddev_val_4','stddev_val_5','stddev_val_6','stddev_val_7','stddev_val_8','stddev_val_9','stddev_val_10','stddev_val_11','stddev_val_12','stddev_val_13','stddev_val_14','stddev_val_15','stddev_val_16','stddev_val_17','stddev_val_18','stddev_val_19']
    step_oht_col = ['STEP_2','STEP_8','STEP_9','STEP_11','STEP_17','STEP_19','STEP_20','STEP_21','STEP_24','STEP_25','STEP_26','STEP_27','STEP_28','STEP_34','STEP_36','STEP_38']
    standard_col=['median_val_3', 'mean_val_8', 'stddev_val_15', 'min_val_12', 'min_val_14','median_val_8','max_val_12', 'rep_hours', 'mean_val_9', 'mean_val_3', 'median_val_2', 'median_val_18', 'mean_val_4', 'median_val_19', 'min_val_13', 'range_val_14', 'max_val_18', 'mean_val_15', 'min_val_3', 'median_val_12','mean_val_19', 'min_val_9', 'max_val_13', 'range_val_13', 'median_val_14', 'mean_val_12', 'mean_val_14', 'mean_val_7', 'min_val_15', 'stddev_val_13', 'mean_val_13', 'max_val_14', 'stddev_val_14', 'median_val_9', 'median_val_13', 'min_val_1',  'max_val_15', 'mean_val_16', 'mean_val_18',  'min_val_16', 'last_hours', 'min_val_19', 'max_val_9', 'stddev_val_12', 'median_val_7', 'max_val_8', 'median_val_4', 'min_val_4', 'median_val_15', 'mean_val_2', 'step_hours', 'stddev_val_9', 'min_val_18', 'median_val_1','max_val_19', 'mean_val_1', 'min_val_8', 'range_val_15', 'max_val_16']
    drop_col = ['STEP_NM','TA_tm_before','start_dt_tm']
    diff_col = ['median_val_1_Diff','max_val_12_Diff','min_val_4_Diff','median_val_3_Diff','min_val_18_Diff','mean_val_9_Diff','mean_val_4_Diff','min_val_3_Diff','mean_val_15_Diff','max_val_15_Diff','range_val_13_Diff','mean_val_13_Diff','stddev_val_13_Diff','max_val_13_Diff','min_val_9_Diff','median_val_7_Diff','mean_val_1_Diff','median_val_14_Diff','stddev_val_9_Diff','max_val_18_Diff','median_val_13_Diff','mean_val_3_Diff','max_val_19_Diff', 'max_val_9_Diff','median_val_4_Diff','mean_val_14_Diff','median_val_18_Diff','min_val_1_Diff','mean_val_2_Diff', 'mean_val_7_Diff','median_val_15_Diff','min_val_13_Diff','stddev_val_15_Diff','max_val_14_Diff','mean_val_12_Diff','median_val_9_Diff','median_val_12_Diff','median_val_2_Diff', 'mean_val_18_Diff', 'range_val_15_Diff','min_val_15_Diff']
    target = ['diff_hours']

    standard_li,step_li,target_li,diff_li = [],[],[],[]
    for i in df.columns:
        if i in standard_col:
            standard_li.append(i)
        elif i in step_oht_col:
            step_li.append(i)
        elif i in target:
            target_li.append(i)
        elif i in diff_col:
            diff_li.append(i)
    new_li = drop_col+step_li+standard_li+target_li
    df = df[new_li]

    print(new_li)
    y = df[target]
    X = df.drop(columns=target)
    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)
    return X,y

In [ ]:
standard_col=['median_val_3', 'mean_val_8', 'stddev_val_15', 'min_val_12', 'min_val_14','median_val_8','max_val_12', 'rep_hours', 'mean_val_9', 'mean_val_3', 'median_val_2', 'median_val_18', 'mean_val_4', 'median_val_19', 'min_val_13', 'range_val_14', 'max_val_18', 'mean_val_15', 'min_val_3', 'median_val_12','mean_val_19', 'min_val_9', 'max_val_13', 'range_val_13', 'median_val_14', 'mean_val_12', 'mean_val_14', 'mean_val_7', 'min_val_15', 'stddev_val_13', 'mean_val_13', 'max_val_14', 'stddev_val_14', 'median_val_9', 'median_val_13', 'min_val_1',  'max_val_15', 'mean_val_16', 'mean_val_18',  'min_val_16', 'last_hours', 'min_val_19', 'max_val_9', 'stddev_val_12', 'median_val_7', 'max_val_8', 'median_val_4', 'min_val_4', 'median_val_15', 'mean_val_2', 'step_hours', 'stddev_val_9', 'min_val_18', 'median_val_1','max_val_19', 'mean_val_1', 'min_val_8', 'range_val_15', 'max_val_16']
diff_col = ['median_val_1_Diff','max_val_12_Diff','min_val_4_Diff','median_val_3_Diff','min_val_18_Diff','mean_val_9_Diff','mean_val_4_Diff','min_val_3_Diff','mean_val_15_Diff','max_val_15_Diff','range_val_13_Diff','mean_val_13_Diff','stddev_val_13_Diff','max_val_13_Diff','min_val_9_Diff','median_val_7_Diff','mean_val_1_Diff','median_val_14_Diff','stddev_val_9_Diff','max_val_18_Diff','median_val_13_Diff','mean_val_3_Diff','max_val_19_Diff', 'max_val_9_Diff','median_val_4_Diff','mean_val_14_Diff','median_val_18_Diff','min_val_1_Diff','mean_val_2_Diff', 'mean_val_7_Diff','median_val_15_Diff','min_val_13_Diff','stddev_val_15_Diff','max_val_14_Diff','mean_val_12_Diff','median_val_9_Diff','median_val_12_Diff','median_val_2_Diff', 'mean_val_18_Diff', 'range_val_15_Diff','min_val_15_Diff']
print(len(standard_col)+len(diff_col))

100


##Metric

In [ ]:
def weighted_rmse1(y_true, y_pred):
    np.random.seed(42)
    tf.keras.utils.set_random_seed(42)
    diff = tf.square(y_true - y_pred)
    rul_t = 1 - (y_true / tf.reduce_max(y_true))
    n_metric = diff * rul_t
    return (tf.reduce_sum(n_metric) / tf.reduce_sum(rul_t)) ** 0.5

def weighted_rmse2(y_true, y_pred):
    np.random.seed(42)
    tf.keras.utils.set_random_seed(42)
    diff = tf.square(y_true - y_pred)
    rul_t = 1 / (y_true+1)
    weighted_diff = diff * rul_t
    weighted_rmse = tf.sqrt(tf.reduce_sum(weighted_diff) / tf.reduce_sum(rul_t))
    return weighted_rmse

def custom_loss(y_true, y_pred):
    np.random.seed(42)
    tf.keras.utils.set_random_seed(42)
    diff = tf.square(y_true - y_pred)
    rul_t = 1 / (y_true+1)
    weighted_diff = diff * rul_t
    weighted_rmse = tf.reduce_sum(weighted_diff)/tf.reduce_sum(rul_t)
    return weighted_rmse

## 모델 예측 후 마지막에 모델 선정


In [ ]:
#base
X,y = xy_split1(df)
#모델
lookback = 25
model = Sequential([
LSTM(128, input_shape=(lookback, len(standard_li)+len(step_li)+len(diff_li)), return_sequences=True),
# Dropout(0.2),
LSTM(64, return_sequences=True),
# Dropout(0.2),
LSTM(32),
# Dropout(0.2),
Dense(1)
])

model.compile(loss=custom_loss, optimizer='adam', metrics=[weighted_rmse1,weighted_rmse2])
# Xy split함수 적용

model1 = None
for x in range(X['TA_tm_before'].nunique()-1):
    scaler = StandardScaler()
    print(f'{x+1}번째 f to f 실행')
    k = x - 6
    if k <= 0:
        k = 0
    print(f'[{k},{x}]')

    X_Train = X.loc[X[X['TA_tm_before'] == X['TA_tm_before'].unique()[k]].head(1).index.item():X[X['TA_tm_before'] == X['TA_tm_before'].unique()[x]].tail(1).index.item(), :]
    for i in X_Train['STEP_NM'].unique():
            for j in standard_li:
                X_Train.loc[X_Train['STEP_NM'] == i ,f'{j}_MA']=X_Train.loc[X_Train['STEP_NM'] == i,j].rolling(window=20,min_periods = 1).mean()
    original_columns = [cols for cols in X_Train.columns if not str(cols).endswith('_MA')] #MA로 끝나지 않는 컬럼들
    for cols in original_columns:
        ma_col = f"{cols}_MA"  # 이동평균 열 이름
        diff_col = f"{cols}_Diff"  # 차이 열 이름

        if ma_col in X_Train.columns:  # 이동평균 열이 존재하는 경우에만 계산
            X_Train[diff_col] = X_Train[cols] - X_Train[ma_col]
    filtered_columns = [colo for colo in X_Train.columns if not str(colo).endswith('_MA')]
    X_Train = X_Train[filtered_columns]
    X_Train = X_Train[drop_col+step_li+standard_li+diff_li]

    X_Train[standard_li+diff_li] = scaler.fit_transform(X_Train[standard_li+diff_li])
    Y_Train = y[y.index.isin(X_Train.index)]
    X_Test = X[X['TA_tm_before'] == X['TA_tm_before'].unique()[x+1]]
    for i in X_Test['STEP_NM'].unique():
            for j in standard_li:
                X_Test.loc[X_Test['STEP_NM'] == i ,f'{j}_MA']=X_Test.loc[X_Test['STEP_NM'] == i,j].rolling(window=20,min_periods = 1).mean()
    original_columns = [cols for cols in X_Test.columns if not str(cols).endswith('_MA')] # 복사본 중에 MA로 끝나지 않는 컬럼들
    for cols in original_columns:
        ma_col = f"{cols}_MA"  # 이동평균 열 이름
        diff_col = f"{cols}_Diff"  # 차이 열 이름

        if ma_col in X_Test.columns:  # 이동평균 열이 존재하는 경우에만 계산
            X_Test[diff_col] = X_Test[cols] - X_Test[ma_col]
    filtered_columns = [colo for colo in X_Test.columns if not str(colo).endswith('_MA')]
    X_Test = X_Test[filtered_columns]
    X_Test = X_Test[drop_col+step_li+standard_li+diff_li]
    # 정규화
    X_Test[standard_li+diff_li] = scaler.transform(X_Test[standard_li+diff_li])

    Y_Test = y[y.index.isin(X_Test.index)]
    X_Train.reset_index(drop=True, inplace=True)
    Y_Train.reset_index(drop=True, inplace=True)
    X_Test.reset_index(drop=True, inplace=True)
    Y_Test.reset_index(drop=True, inplace=True)
    print(X_Train.shape,Y_Train.shape,X_Test.shape,Y_Test.shape)

    # 모델 학습
    if len(X_Train) > lookback:
        x_trains, y_trains = [], []

        for i in X_Train['TA_tm_before'].unique():
            for j in range(len(X_Train[X_Train['TA_tm_before'] == i]) - lookback):
                X_train = X_Train[X_Train['TA_tm_before'] == i].iloc[j:j + lookback, :].drop(columns=['STEP_NM', 'TA_tm_before', 'start_dt_tm'])
                x_trains.append(X_train)
                y_index = X_train.index[-1]  # X_train의 마지막 행의 인덱스 가져오기
                Y_train = Y_Train.loc[y_index]  # 해당 인덱스에 해당하는 Y_Train 값 가져오기
                y_trains.append(Y_train)
        x_train, y_train = np.array(x_trains), np.array(y_trains)
        print(len(x_train),len(y_train))
        np.random.seed(42)
        tf.keras.utils.set_random_seed(42)
        print('start_train')
        model.fit(x_train, y_train, epochs=30, batch_size=50, verbose=1)
        print('end_train')
        model.save(f'/content/drive/MyDrive/models/model_{x+1}.h5')
        print(f'{x}번 모델 저장 완료')
    if len(X_Test) > lookback:
        x_tests, y_tests = [], []
        for i in range(len(X_Test) - lookback):
            X_test = X_Test.iloc[i:i + lookback, :].drop(columns=['STEP_NM', 'TA_tm_before', 'start_dt_tm'])
            Y_test = Y_Test.iloc[i + lookback-1]
            x_tests.append(X_test)
            y_tests.append(Y_test)
        x_test, y_test = np.array(x_tests), np.array(y_tests)

        if model1 is not None:
            model1_score = weighted_rmse2(y_train.reshape(-1), model1.predict(x_train).reshape(-1))
            print('model1이 존재함')
            testPredict = model.predict(x_test).reshape(-1)
            testPredict1 = model1.predict(x_test).reshape(-1)

        else:
            print('아직 model1이 존재하지 않음')
            np.random.seed(42)
            tf.keras.utils.set_random_seed(42)
            testPredict = model.predict(x_test).reshape(-1)



        testPredict = pd.Series(testPredict)
        testPredict = testPredict.rolling(window=50, min_periods=1).mean()
        for i in range(0, len(testPredict) - 1):
            if testPredict[i + 1] > testPredict[i]:
                testPredict[i + 1] = testPredict[i]
        testPredict[testPredict < 0] = 0
        mse_score = mean_squared_error(y_test.reshape(-1), testPredict)
        wrmse1 = weighted_rmse1(y_test.reshape(-1), testPredict)
        wrmse2 = weighted_rmse2(y_test.reshape(-1), testPredict)


        print(f'model_mse_score : {mse_score}, model_wrmse1 : {wrmse1}, model_wrmse2 : {wrmse2}')
        try:
            testPredict1 = pd.Series(testPredict1)
            testPredict1 = testPredict1.rolling(window=50, min_periods=1).mean()
            for i in range(0, len(testPredict1) - 1):
                if testPredict1[i + 1] > testPredict1[i]:
                    testPredict1[i + 1] = testPredict1[i]
            testPredict1[testPredict1 < 0] = 0

            # testPredict와 testPredict1 중에서 작은 값을 선택하여 새로운 시리즈로 만듭니다.
            ensemble = pd.concat([testPredict, testPredict1], axis=1).min(axis=1)
            ensemble_mse_score = mean_squared_error(y_test.reshape(-1), ensemble)
            ensemble_wrmse1 = weighted_rmse1(y_test.reshape(-1), ensemble)
            ensemble_wrmse2 = weighted_rmse2(y_test.reshape(-1), ensemble)
            print(f'model_mse_score : {ensemble_mse_score}, model_wrmse1 : {ensemble_wrmse1}, model_wrmse2 : {ensemble_wrmse2}')

        except Exception as e:
            pass
        X_Test['start_dt_tm'] = pd.to_datetime(X_Test['start_dt_tm'])
        X_Test.sort_values('start_dt_tm', inplace=True)

        # plot하기
        plt.figure(figsize=(15, 5))
        plt.plot(X_Test.loc[lookback:,'start_dt_tm'], testPredict, marker='o', linestyle='-', label='model pred', alpha=0.5)
        try:
            plt.plot(X_Test.loc[lookback:,'start_dt_tm'], testPredict1, marker='o', linestyle='-', label='model1 pred', alpha=0.5)
            plt.plot(X_Test.loc[lookback:,'start_dt_tm'], ensemble, marker='o', linestyle='-', label='ensemble', alpha=0.5)
        except:
            pass
        plt.plot(X_Test.loc[lookback:,'start_dt_tm'], Y_Test[lookback:], marker='o', linestyle='-', label='True values', alpha=0.5)
        plt.title('Diff Hours Over Time')
        plt.xlabel('Start Date Time')
        plt.ylabel('Diff Hours')
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

    else:
        pass


    model_dir = '/content/drive/MyDrive/models/'
    Predict_val = {}
    pred_score = {}
    for filename in os.listdir(model_dir):
        if filename.startswith("model_") and filename.endswith(".h5"):
            print(filename)
            model_path = os.path.join(model_dir, filename)
            model_s3 = load_model(model_path, custom_objects={'custom_loss': custom_loss, 'weighted_rmse1': weighted_rmse1, 'weighted_rmse2': weighted_rmse2})
            s3_pred=model_s3.predict(x_test).reshape(-1)
            Predict_val[filename] = s3_pred
            pred_score[filename] = mean_squared_error(y_test.reshape(-1), s3_pred) # weighted mean squared error로 변경 필요!!!!!
            print(filename, pred_score[filename])
    top_2_score = heapq.nsmallest(5, pred_score, key=pred_score.get)
    print(top_2_score)
    model1 = load_model(f'/content/drive/MyDrive/models/{top_2_score[0]}', custom_objects={'custom_loss': custom_loss, 'weighted_rmse1': weighted_rmse1, 'weighted_rmse2': weighted_rmse2})
    if x!= 0:
        data = {'start_dt_tm': X_Test.loc[lookback:,'start_dt_tm'],'model_pred':testPredict,'ensemble':ensemble}
        df1 = pd.DataFrame(data)
        df1.to_csv(f'/content/drive/MyDrive/df_result/model_{x}_result.csv')

Output hidden; open in https://colab.research.google.com to view.